<a href="https://colab.research.google.com/github/LucianaNieto/Cambodia_2023/blob/main/image_chips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

#Image Chips for image segmentation  

### Import the API

Run the following cell to import the API into your session.

In [ ]:
import ee
!pip install retry


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=V5XQ29CKnQCYBBDnuoPtrdDC0owDv_-ZIJ3Ec3viZpE&tc=AWd0cbh8SLcQjYLBfgCYRy3K7quKBngsBoi0rDbx5vM&cc=UxEuJm559t5X3SDA5IBm-oKe-bOS-AuC99p-3znIu94

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh5WewsmrVuUABlbhW_P5UJa0dBUhGEEFce-kQk0JMsXeKmRtFcoLuE

Successfully saved authorization token.


In [ ]:

import logging
import multiprocessing
import requests
import shutil
from retry import retry


"""
This tool demonstrates extracting data from Earth Engine using parallel
request and getThumbURL.
"""

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
def getRequests():
  """Generates a list of work items to be downloaded.

  Produces 1000 random points in each of the RESOLVE ecoregions in the ROI.
  """
  # Get our ROI from the GAUL regions
  gaul = ee.FeatureCollection('projects/phenology-052020/assets/cambodia_2020_13E')

  area = list([(103.26771881167686,13.076117576456296),
              (103.37414875641629,13.075672001895128),
              (103.3607306545007,13.022642889940643),
               (103.268633682262,13.022791446747856),
               (103.26771881167686,13.076117576456296)])
  
  
  roi = ee.Geometry.Polygon(area)
  #roi = gaul.filter('ADM1_NAME == "Colorado"').first().geometry()

  # To stratify by RESOLVE ecoregion, paint the ECO_IDs into an image.
  resolve = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')
  ecoregions = (resolve.reduceToImage(['ECO_ID'], ee.Reducer.first())
                .clip(roi).rename('ECO_ID'))
  points = ecoregions.stratifiedSample(
      numPoints=500,
      classBand='ECO_ID',
      region=roi,
      scale=100,
      geometries=True)

  return points.aggregate_array('.geo').getInfo()


@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
  """Handle the HTTP requests to download an image."""

  # Generate the desired image from the given point.
  point = ee.Geometry.Point(point['coordinates'])
  region = point.buffer(150).bounds() #254 overall 
  image = (ee.ImageCollection('projects/phenology-052020/assets/cambodia_2020_13E')
           .filterBounds(region)
           .filterDate('2020-03-05', '2020-03-06')
           .mosaic()
           .clip(region)
           .select('B3', 'B2', 'B1'))
           #.select('B4'))

  # Fetch the URL from which to download the image.
  url = image.getDownloadURL({
      'region': region,
      'dimensions': '512x512',
      'format': 'GeoTIFF'})

  # Handle downloading the actual pixels.
  r = requests.get(url, stream=True)
  if r.status_code != 200:
    r.raise_for_status()

  #filename = 'tile_%05d.png' % index
  filename = '/content/drive/MyDrive/roi_06_05/ROI_6_05_%05d.tiff' % index
  with open(filename, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)
  print("Done: ", index)


if __name__ == '__main__':
  logging.basicConfig()
  items = getRequests()

  pool = multiprocessing.Pool(25)
  pool.starmap(getResult, enumerate(items))

  pool.close()

Done:  Done: Done: 150  
16040

Done:  20
Done:  10
Done:  151
Done:  21
Done:  120
Done:  11
Done:  230
Done: Done:   210Done: 41 

161
Done: Done:   190152Done: Done: Done: 
Done: Done: 
   200
121 100 

220110Done:  

Done: Done: 231  
5012
Done: Done: 
  1400

Done:  180
Done:  70
Done:  240Done: Done: 
 170
 Done: 80
 60Done: Done:  
 130
162
Done:  22
Done:  90
Done:  Done: 211 Done: Done:   19130


101
Done:  51
Done:  13
Done:  42
Done:  71
Done:  232
Done: Done:   Done:  153141Done: 201
Done: 
  
111Done: 122Done:   
131Done: 

 6123Done:  

1Done: Done:  
212 
Done: 181Done: Done: Done:  
  102 171Done: 
Done: 192
81 
 22172


Done:  14
Done:  163Done: 
Done:  91 
31Done:  Done: 
 112202

Done: Done:   142154

Done:  123
Done: Done:  Done: 241  24

52
Done: Done: Done:    Done: 2 182

233213Done: 
 
43
Done:  32
Done:  92
Done:  62
Done:  73
Done:  103
Done:  172
Done:  82
Done:  25
Done:  15
Done:  132
Done:  53
Done: Done:  164
Done:  155 
234
Done:  203
Done: Done: Done:  

In [ ]:
import os
import imageio

yourpath = '/content/drive/MyDrive/roi_06_05'
for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
        print(os.path.join(root, name))
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tiff":
            if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                print ("A jpeg file already exists for %s" % name)
            # If a jpeg is *NOT* present, create one from the tiff.
            else:
                outfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                try:
                    im = imageio.imread(os.path.join(root, name))
                    print ("Generating jpeg for %s" % name)
                    imageio.imsave(outfile, im, format='JPEG')
                except Exception as e:
                    print(e)


/content/drive/MyDrive/roi_06_05/ROI_6_05_00150.tiff
Generating jpeg for ROI_6_05_00150.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00160.tiff
Generating jpeg for ROI_6_05_00160.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00040.tiff
Generating jpeg for ROI_6_05_00040.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00020.tiff
Generating jpeg for ROI_6_05_00020.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00010.tiff
Generating jpeg for ROI_6_05_00010.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00151.tiff
Generating jpeg for ROI_6_05_00151.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00021.tiff
Generating jpeg for ROI_6_05_00021.tiff


/content/drive/MyDrive/roi_06_05/ROI_6_05_00120.tiff
Generating jpeg for ROI_6_05_00120.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00011.tiff
Generating jpeg for ROI_6_05_00011.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00230.tiff
Generating jpeg for ROI_6_05_00230.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00041.tiff
Generating jpeg for ROI_6_05_00041.tiff


/content/drive/MyDrive/roi_06_05/ROI_6_05_00210.tiff


Generating jpeg for ROI_6_05_00210.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00161.tiff
Generating jpeg for ROI_6_05_00161.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00152.tiff
Generating jpeg for ROI_6_05_00152.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00190.tiff
Generating jpeg for ROI_6_05_00190.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00200.tiff
Generating jpeg for ROI_6_05_00200.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00100.tiff
Generating jpeg for ROI_6_05_00100.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00121.tiff
Generating jpeg for ROI_6_05_00121.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00110.tiff
Generating jpeg for ROI_6_05_00110.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00220.tiff
Generating jpeg for ROI_6_05_00220.tiff


/content/drive/MyDrive/roi_06_05/ROI_6_05_00231.tiff
Generating jpeg for ROI_6_05_00231.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00012.tiff
Generating jpeg for ROI_6_05_00012.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00050.tiff
Generating jpeg for ROI_6_05_00050.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00140.tiff
Generating jpeg for ROI_6_05_00140.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00000.tiff
Generating jpeg for ROI_6_05_00000.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00180.tiff
Generating jpeg for ROI_6_05_00180.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00070.tiff
Generating jpeg for ROI_6_05_00070.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00240.tiff
Generating jpeg for ROI_6_05_00240.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00170.tiff
Generating jpeg for ROI_6_05_00170.tiff


...

/content/drive/MyDrive/roi_06_05/ROI_6_05_00927.tiff
Generating jpeg for ROI_6_05_00927.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00888.tiff
Generating jpeg for ROI_6_05_00888.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00957.tiff
Generating jpeg for ROI_6_05_00957.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00996.tiff
Generating jpeg for ROI_6_05_00996.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00859.tiff
Generating jpeg for ROI_6_05_00859.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00868.tiff
Generating jpeg for ROI_6_05_00868.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00839.tiff
Generating jpeg for ROI_6_05_00839.tiff


/content/drive/MyDrive/roi_06_05/ROI_6_05_00977.tiff
Generating jpeg for ROI_6_05_00977.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00878.tiff
Generating jpeg for ROI_6_05_00878.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00937.tiff
Generating jpeg for ROI_6_05_00937.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00918.tiff
Generating jpeg for ROI_6_05_00918.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00819.tiff
Generating jpeg for ROI_6_05_00819.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00908.tiff
Generating jpeg for ROI_6_05_00908.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00947.tiff
Generating jpeg for ROI_6_05_00947.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00899.tiff
Generating jpeg for ROI_6_05_00899.tiff


/content/drive/MyDrive/roi_06_05/ROI_6_05_00987.tiff
Generating jpeg for ROI_6_05_00987.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00967.tiff
Generating jpeg for ROI_6_05_00967.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00849.tiff
Generating jpeg for ROI_6_05_00849.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00889.tiff
Generating jpeg for ROI_6_05_00889.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00869.tiff
Generating jpeg for ROI_6_05_00869.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00928.tiff
Generating jpeg for ROI_6_05_00928.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00958.tiff
Generating jpeg for ROI_6_05_00958.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00997.tiff
Generating jpeg for ROI_6_05_00997.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00938.tiff


Generating jpeg for ROI_6_05_00938.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00978.tiff
Generating jpeg for ROI_6_05_00978.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00919.tiff
Generating jpeg for ROI_6_05_00919.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00909.tiff
Generating jpeg for ROI_6_05_00909.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00948.tiff
Generating jpeg for ROI_6_05_00948.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00988.tiff
Generating jpeg for ROI_6_05_00988.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00968.tiff
Generating jpeg for ROI_6_05_00968.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00879.tiff
Generating jpeg for ROI_6_05_00879.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00929.tiff
Generating jpeg for ROI_6_05_00929.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00959.tiff
Generating jpeg for ROI_6_05_00959.tiff


/content/drive/MyDrive/roi_06_05/ROI_6_05_00979.tiff
Generating jpeg for ROI_6_05_00979.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00998.tiff
Generating jpeg for ROI_6_05_00998.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00939.tiff
Generating jpeg for ROI_6_05_00939.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00989.tiff
Generating jpeg for ROI_6_05_00989.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00949.tiff
Generating jpeg for ROI_6_05_00949.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00969.tiff
Generating jpeg for ROI_6_05_00969.tiff
/content/drive/MyDrive/roi_06_05/ROI_6_05_00999.tiff
Generating jpeg for ROI_6_05_00999.tiff
